#### this notebook is for using scipy curve\_fit to tweak parameters in a known form

In [86]:
from pysr import PySRRegressor, TemplateExpressionSpec

import matplotlib.pyplot as plt
import numpy as np

import camb

from scipy.optimize import curve_fit

In [87]:
# Load Data
pars     = np.load('CL_data/parameter_test9.npy')  # [H0, ombh2, omch2 ] x 100
lensed   = np.load('CL_data/lensed_CL9.npy')     # [C_2, ..., C_5000] x 100 (lensed)
unlensed = np.load('CL_data/unlensed_CL9.npy')     # [C_2, ..., C_5000] x 100 (unlensed)

In [88]:
#extracting info for independent variables
ombs = pars[:,1]
omcs = pars[:,2]

ells = np.array([l for l in range(2, 5000)])

In [89]:
#getting dependent variables
lensing = lensed/unlensed #lensing tail
camb_data_format = []
for i in range(len(lensing)):
    for j in range(len(lensing[i])):
        camb_data_format.append(lensing[i][j])

In [92]:
#the fidicual values for normalizing variables
fid_ombh2 = 0.0224
fid_omch2 = 0.12

In [121]:
def func(X, shift1, shift2):
    #expected form of the lensing tail function for curve fitting
    ell = X[0]
    ommh2 = X[1]
    beta3 = 3776 * np.exp((ommh2/0.1424) * shift1)
    beta4 = 341 * np.exp((ommh2/0.1424) * shift2)
    alpha = (ommh2/0.1414)/0.4  #keeping this fixed
    sigma = (1 + np.exp(-(ell - beta3)/beta4))**-1
    poly = (0.7 * (ell/2800)**alpha -1)
    
    return (poly*sigma + 1).flatten() 

In [122]:
cosmos = [ombs[i] + omcs[i] for i in range(len(ombs))]    #x3 variable
n = len(cosmos) #number of cosmologies
params = np.zeros((len(ells) * n, 2))   #ind variables
for i in range(n):
    params[(i)*len(ells):(i+1)*len(ells),0] = ells
for i in range(n):
    params[(i)*len(ells):(i+1)*len(ells),1] = cosmos[i]  #ommh2

In [123]:
p0 = 0,0 #initial values

In [124]:
x1 = params[:,0]
x2 = params[:,1]

In [126]:
answers, _ = curve_fit(func,(x1, x2),camb_data_format, p0)  #doing the curve fitting

In [127]:
answers

array([-0.01296149,  0.3473414 ])